In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from cmath import nan
import pywt
import torch
import pywt
from pandarallel import pandarallel
import tqdm
from functools import partial
import glob
from pathlib import Path

In [ ]:
def data_split(input_data, data_field):
    #Data split by 0.001 Sec
    up = float(Data['Time_Ofset'][0])
    low = up + 10
    wave_out = pd.DataFrame(np.zeros(23)).T
    output = []
    for i in np.array(Data.Time_Ofset):
        if  float(i) <= low:
            out = Data[(Data['Time_Ofset'] >= float(up)) & (Data['Time_Ofset'] <= float(low))]
            if len(out) == 0:
                break
            else:
                output.append(np.array(out[[data_field]]))

        up = low
        low = low + 10

    return np.array(output)

In [ ]:
def convert_cwt_wavelet(Data_input):
    wave_out = pd.DataFrame(np.zeros(23)).T
    for i in Data_input:
        coef, freqs = pywt.cwt(i,np.arange(1,129),'gaus1')
        out = pd.concat([pd.DataFrame(np.reshape(coef,(coef.shape[0], coef.shape[1]))), pd.DataFrame(freqs)], 
                            axis=1, ignore_index=True).fillna(0)
        cwt_wave_out = pd.concat([wave_out,out], ignore_index=True).fillna(0)
    return cwt_wave_out

In [ ]:
def convert_dwt_wavelet(Data_input, Wvaelet_Family, Label, Mode):
    wave_out = []

    for i in Data_input:
        a, b, c = pywt.wavedec(i, Wvaelet_Family, level = Label, mode = Mode)
        wave_out.append(pd.DataFrame((a.flatten(), b.flatten(), c.flatten())).T)

    join = wave_out[0]
    for z in wave_out:
        join = pd.concat([join, z])
        dwt_wave_out = join[1:]
    return dwt_wave_out

In [ ]:
path = 'Labeled data/'
# get the files from the path provided in the OP
files = Path(path).glob('*.csv')  # .rglob to get subdirectories
csv_files = glob.glob(path + "/*.csv")
pd.DataFrame(csv_files, columns=['Data List'])

In [ ]:
dfs = list()
for f in files:
    data = pd.read_csv(f)
    # .stem is method for pathlib objects to get the filename w/o the extension
    data['File'] = f.stem
    dfs.append(data)

In [ ]:
Data = dfs[0]
Data

In [ ]:
Data = dfs[12]
print("Column Names: \n\n", Data.columns)

In [ ]:
data_field = 'Scaled_CAN_ID'
data_loading = data_split(Data, data_field)
data_loading.shape

In [ ]:
wavelet_set = convert_cwt_wavelet(data_loading[1])
wavelet_set

In [ ]:
Input_Data = data_loading
Wvaelet_Family = 'db2'
Label = 2
Mode = 'periodic'

wavelet_set = convert_dwt_wavelet(Input_Data, Wvaelet_Family, Label, Mode)
wavelet_set